## School Admission
NN that should decide schools admissions
### Inputs:
* GRD Score
* GPA
* The rank of the school, it's a category

### Data cleanup
* we standerize GRD and GPA from -1.0 to 1.0. It's necessary because the sigmoid function squashes really small and really large inputs.
* the rank we use dummy varibles as inputs since it has not any relation its values.
* we split off randomly a 10% of the data for TESTING

```
admissions = pd.read_csv('datasets/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
targets_test.T
```

In [111]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('datasets/binary.csv')
admissions.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
admit,0.00,1.00,1.0,1.00,0.00,1.0,1.00,0.00,1.00,0.00,...,1.00,1.00,1.00,1.00,1.00,0.0,0.00,0.00,0.00,0.00
gre,380.00,660.00,800.0,640.00,520.00,760.0,560.00,400.00,540.00,700.00,...,800.00,660.00,600.00,620.00,460.00,620.0,560.00,460.00,700.00,600.00
gpa,3.61,3.67,4.0,3.19,2.93,3.0,2.98,3.08,3.39,3.92,...,3.05,3.88,3.38,3.75,3.99,4.0,3.04,2.63,3.65,3.89
rank,3.00,3.00,1.0,4.00,4.00,2.0,1.00,2.00,3.00,2.00,...,2.00,2.00,3.00,2.00,3.00,2.0,3.00,2.00,2.00,3.00


### Convert category input 'rank' to dummy variables

In [112]:
# make dummy varianles for rank
dummy_rank = pd.get_dummies(admissions['rank'], prefix='rank')
dummy_rank.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
rank_1,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rank_2,0,0,0,0,0,1,0,1,0,1,...,1,1,0,1,0,1,0,1,1,0
rank_3,1,1,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,1,0,0,1
rank_4,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
# remove rank
admissions = admissions.drop('rank', axis=1)
admissions.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
admit,0.00,1.00,1.0,1.00,0.00,1.0,1.00,0.00,1.00,0.00,...,1.00,1.00,1.00,1.00,1.00,0.0,0.00,0.00,0.00,0.00
gre,380.00,660.00,800.0,640.00,520.00,760.0,560.00,400.00,540.00,700.00,...,800.00,660.00,600.00,620.00,460.00,620.0,560.00,460.00,700.00,600.00
gpa,3.61,3.67,4.0,3.19,2.93,3.0,2.98,3.08,3.39,3.92,...,3.05,3.88,3.38,3.75,3.99,4.0,3.04,2.63,3.65,3.89


In [114]:
# concatenate rank dummy variables
data = pd.concat([admissions, dummy_rank], axis=1)
data.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
admit,0.00,1.00,1.0,1.00,0.00,1.0,1.00,0.00,1.00,0.00,...,1.00,1.00,1.00,1.00,1.00,0.0,0.00,0.00,0.00,0.00
gre,380.00,660.00,800.0,640.00,520.00,760.0,560.00,400.00,540.00,700.00,...,800.00,660.00,600.00,620.00,460.00,620.0,560.00,460.00,700.00,600.00
gpa,3.61,3.67,4.0,3.19,2.93,3.0,2.98,3.08,3.39,3.92,...,3.05,3.88,3.38,3.75,3.99,4.0,3.04,2.63,3.65,3.89
rank_1,0.00,0.00,1.0,0.00,0.00,0.0,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
rank_2,0.00,0.00,0.0,0.00,0.00,1.0,0.00,1.00,0.00,1.00,...,1.00,1.00,0.00,1.00,0.00,1.0,0.00,1.00,1.00,0.00
rank_3,1.00,1.00,0.0,0.00,0.00,0.0,0.00,0.00,1.00,0.00,...,0.00,0.00,1.00,0.00,1.00,0.0,1.00,0.00,0.00,1.00
rank_4,0.00,0.00,0.0,1.00,1.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00


### Standarize features 'gre' and 'gpa'

In [115]:
# standarize features
for field in ['gre', 'gpa']:
    mean = data[field].mean()
    std = data[field].std()
    data.loc[:, field] = (data[field] -  mean) / std
data.T

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
admit,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gre,-1.798011,0.625884,1.837832,0.452749,-0.586063,1.491561,-0.239793,-1.624876,-0.412928,0.972155,...,1.837832,0.625884,0.106478,0.279614,-1.105469,0.279614,-0.239793,-1.105469,0.972155,0.106478
gpa,0.578348,0.736008,1.603135,-0.525269,-1.208461,-1.024525,-1.077078,-0.814312,0.000263,1.392922,...,-0.893142,1.287816,-0.026014,0.946220,1.576859,1.603135,-0.919418,-1.996759,0.683454,1.314093
rank_1,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rank_2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
rank_3,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
rank_4,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### We split off randomly 10% of the data for testing

In [116]:
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data) * 0.9), replace=False)
# multime-statement does NOT modify data object
data, test_data = data.iloc[sample], data.drop(sample)
## WARNING!! separated statements don't work
## because the 1st statement modifies data, we should use an aux variable in this case
# data = data.iloc(sample)
# test_data = data.drop(sample)
test_data.T

,20,21,48,50,54,58,71,87,99,102,...,308,313,330,343,344,348,359,372,397,399
admit,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
gre,-0.759199,0.625884,-1.278605,0.452749,0.625884,-1.624876,-2.490553,0.106478,-1.624876,-1.798011,...,-0.412928,-0.586063,1.318426,-0.066657,-0.586063,-1.624876,1.145291,0.799020,-1.105469,0.106478
gpa,-0.577822,0.630901,-2.390908,1.235263,-0.131120,0.683454,-1.234737,0.236752,-0.209950,-0.157397,...,-0.682929,0.683454,1.603135,-0.866865,-0.104844,-0.078567,1.445476,-2.548567,-1.996759,1.314093
rank_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
rank_2,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
rank_3,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
rank_4,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Split into features and targets

In [117]:
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

## Gradient descent and network training

In [118]:

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        h = np.dot(weights, x)
        output = sigmoid(h)

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * output * (1 - output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

('Train loss: ', 0.26276093849966348)
('Train loss: ', 0.20928619409324875)
('Train loss: ', 0.20084292908073426)
('Train loss: ', 0.19862156475527873)
('Train loss: ', 0.1977985139668603)
('Train loss: ', 0.19742577912189863)
('Train loss: ', 0.19723507746241059)
('Train loss: ', 0.19712945625092479)
('Train loss: ', 0.19706766341315082)
('Train loss: ', 0.19703005801777368)
Prediction accuracy: 0.725
